### Scraping main URLs from the web to build the knowledge base

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import fitz

def fetch_text(url):
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--accept-cookies")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")    
    driver = webdriver.Chrome(options=options)

    if url.lower().endswith('.pdf'):
        response = requests.get(url)
        response.raise_for_status()
        
        pdf_text = ""
        with fitz.open(stream=response.content, filetype="pdf") as pdf:
            for page_num in range(pdf.page_count):
                pdf_text += pdf[page_num].get_text("text")
        raw_text = pdf_text

    else:
        driver.get(url)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        raw_text = soup.get_text(separator=' ')
    
    cleaned_text = re.sub(r'[^a-zA-ZáéíóúñÁÉÍÓÚÑüÜ0-9\s,.]', '', raw_text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    cleaned_text = cleaned_text.replace("....", "")
    return cleaned_text.strip()


def generate_dataframe(urls):
    data = []
    for url in urls:
        text = fetch_text(url)
        if text:
            data.append({"url": url, "text": text})
    
    df = pd.DataFrame(data, columns=["url", "text"])
    return df

urls = ["https://smowl.net/es/",
        "https://smowl.net/es/proctoring-empresa/",
        "https://smowl.net/es/soporte-para-examinados/",
        "https://smowl.net/es/proctoring-educacion/",
        "https://smowl.net/es/productos-de-proctoring/",
        "https://smowl.net/es/sobre-nosotros/",
        "https://smowl.net/es/faqs-proctoring/",
        "https://smowl.net/es/faqs-proctoring/usuario/",
        "https://smowl.net/es/soporte-para-examinados/",
        "https://media.smowltech.net/html/privacy/dpa/smowltech_dpa_es.html",
        "https://media.smowltech.net/html/conditions/conditions_SCM_2_es.html",
        "https://media.smowltech.net/html/conditions/conditions_GDPR_2_es.html",
        "https://media.smowltech.net/html/terms/conditions_SMOWLPLUS_2_es.html",
        "https://smowl.net/en/privacy-policy/",
        "https://media.smowltech.net/pdf/guides/smowlcm_userguide_installation_es.pdf"]

df = generate_dataframe(urls)

In [5]:
df

,url,text
0,https://smowl.net/es/,SMOWL Software de supervisión de exámenes onli...
1,https://smowl.net/es/proctoring-empresa/,Proctoring para empresas Smowl Consent Details...
2,https://smowl.net/es/soporte-para-examinados/,Soporte técnico Smowl Skip to content mySmowlt...
3,https://smowl.net/es/proctoring-educacion/,Supervisar exámenes online Smowl Consent Detai...
4,https://smowl.net/es/productos-de-proctoring/,Software de proctoring SMOWL Proctoring Sistem...
5,https://smowl.net/es/sobre-nosotros/,"Smowltech, quiénes somos Consent Details IABV2..."
6,https://smowl.net/es/faqs-proctoring/,Preguntas frecuentes SMOWL Skip to content myS...
7,https://smowl.net/es/faqs-proctoring/usuario/,Información y soporte para usuarios SMOWL Skip...
8,https://smowl.net/es/soporte-para-examinados/,Soporte técnico Smowl Skip to content mySmowlt...
9,https://media.smowltech.net/html/privacy/dpa/s...,Smowltech DPA CONTRATO DE ENCARGADO DE TRATAMI...


### Building a FAISS knowloedge vector database using an open-source multilingual embeddings model

In [7]:
from sentence_transformers import SentenceTransformer
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

model_id = "intfloat/multilingual-e5-large"

model = SentenceTransformer(model_id)
embeddings = model.encode(df["text"])

df['embeddings'] = embeddings.tolist()
loader = DataFrameLoader(df, page_content_column="text")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=model.max_seq_length,
    chunk_overlap=model.max_seq_length //10,
)
docs = text_splitter.split_documents(documents)

/Users/inaki.gorostiaga/Personal/chatbots/venv-chatbot/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from langchain.vectorstores import FAISS
from langchain_core.embeddings import Embeddings


class SentenceTransformerEmbeddings(Embeddings):
    """Embedding model using Sentence Transformers."""

    def __init__(self, model_name: str = ''):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts: list[str]) -> list[list[float]]:
        """Embed a list of documents."""
        return self.model.encode(texts, convert_to_numpy=True).tolist()

    def embed_query(self, text: str) -> list[float]:
        """Embed a single query text."""
        return self.model.encode(text, convert_to_numpy=True).tolist()

vectorstore_faiss = FAISS.from_documents(
    documents=docs,
    embedding=SentenceTransformerEmbeddings(model_id)
)

### Debugging queries

In [13]:
matching_docs = vectorstore_faiss.similarity_search_with_score("Qué es el CM?",k=5)
[i[0].page_content for i in matching_docs]

['Soporte Blog Solicitar demo gratis Solicitar demo gratis Preguntas frecuentes SMOWL Somos transparentes y te damos información detallada Si necesitas hacernos una consulta que no esté en esta página puedes contactar directamente con nuestro equipo de soporte técnico. Contacta aquí Información para instructores Información para instructores Información para usuarios Información para usuarios Qué es SMOWL CM E s una aplicación de escritorio que supervisa la actividad de un usuario durante una prueba online.',
 'desde el enlace proporcionado por nuestra institución. Actualización de CMPuede que una vez que termines con la instalación de CM, o antes de iniciar tu examen o actividad, aparezca una ventana indicando que el sistema se encuentra buscando nuevas actualizaciones de la herramienta.En caso de que existan, te será notificado con el mensaje Actualización disponible.Para actualizar tu versión de SMOWLCM, deberás descargarla y posteriormente instalarla pulsando sobre los botones corr

In [14]:
matching_docs = vectorstore_faiss.similarity_search_with_score("Quien es el CEO de smowl?",k=5)
[i[0].page_content for i in matching_docs]

['europea y por ello cumplimos totalmente con el Reglamento General de Protección de Datos. Además, como recibimos fondos por parte de la Unión Europea, cumplimos con las exigencias de ética y seguridad. Es el momento de la transparencia Sabemos que elegir la herramienta adecuada es importante para ti. Por eso, te damos una demostración de SMOWL y atendemos todas tus preguntas en una reunión para ampliar la información que necesites. Solicitar mi demo gratuita o mi reunión personalizada Sobre Smowltech',
 'Preguntas frecuentes SMOWL Skip to content mySmowltech ES EN Usos Empresa Educación Planes Nosotros Soporte Preguntas frecuentes Estudiante Soporte técnico Blog Menu Usos Empresa Educación Planes Nosotros Soporte Preguntas frecuentes Estudiante Soporte técnico Blog Usos Empresa Educación Planes Smowltech Soporte Preguntas frecuentes Estudiante Soporte Blog Solicitar demo gratis Menu Usos Empresa Educación Planes Smowltech Soporte Preguntas frecuentes Estudiante Soporte Blog Solicitar

In [16]:
matching_docs = vectorstore_faiss.similarity_search_with_score("Como solicito una prueba?",k=5)
[i[0].page_content for i in matching_docs]

['exámenes online Instalación sencilla e intuitiva. Soporte técnico 247 para incidencias. Privacidad del usuario garantizada cumpliendo la RGPD. Supervisión 100 personalizable y escalable. Solicitar una demo gratis 25 licencias gratis Ver una demo grabada Ahorra hasta un 75 en exámenes Más de 200 instituciones en todo el mundo confían en nosotros 8,8 Satisfacción de los usuarios con nuestro soporte Casos prácticos Nuestros clientes te lo cuentan Ver casos de estudio Mientras el alumno realiza el examen,',
 'Una vez que nos escribas te responderemos en el menor tiempo posible por correo electrónico o videollamada. Comunica una incidencia Nuestro producto está basado en la tecnología. Es minucioso y eficaz, pero hasta las máquinas a veces fallan. Si te encuentras alguna anomalía, contacta con nosotros. Pregunta o pide información Tienes alguna duda Compártela con el equipo. Pregúntanos cualquier aspecto relacionado con el funcionamiento de la herramienta. Para contactar con nuestro equip

### Loading the LLM model with Ollama

In [17]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

template = """Question: {question}

Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_template(template)

llm = OllamaLLM(model="llama3.1:8b")

### Constructing the QA chain

In [23]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """You will be acting as a customer support chatbot named Smowlbot, created by the company Smowltech, a leader in online proctoring solutions.  
Your goal is to provide support to customers regarding Smowltech’s products and services, ensuring secure and reliable environments for remote exams and assessments.  
You are part of Smowltech's customer support team. Always stay in character.

Here are some important rules for the interaction:  
- Use the chat history and the following pieces of context to answer the question.  
- Only use relevant parts of the context.  
- If the context is meaningful to answer the query, ensure your response aligns with the context and refrain from fabricating answers.  
- Do not include web, email, or other links unless they are in the context.  
- Provide assistance, recommendations, or advice only on matters related to Smowltech's proctoring technology, services, and processes.  
- Always maintain a friendly and professional tone in your responses.  
- Use three sentences maximum and keep the answer clear, concise, and easy to follow.  
- If the question is unclear or you need additional crucial user information to provide an accurate answer, feel free to ask the user for more details.  

Here is the context:  
<context>  
{context}  
</context>  
Here is the user’s question:  
<question>  
{question}  
</question>  

How do you respond to the user’s question?  
Answer in the language of the question.  
Always focus on responding to the user's messages and using only the context that directly relates to their queries.  
Put your response in <response></response> tags.  
Think about your answer first before you respond.  
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore_faiss.as_retriever(
        search_type="similarity", search_kwargs={"k": 7}
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

### Debugging and fine-tuning the QA chain

In [27]:
output =  qa({"query": "Quien es el CEO de smowl?"})
result = output["result"]
source_documents = output["source_documents"]

print(result)
print([i.metadata["url"] for i in source_documents])

<response>
Hola! Me alegra responder a tu pregunta. Según la información que tenemos en el contexto, Ricardo Vea es el CEO de Smowltech y también profesor universitario. ¡Espero haberte ayudado!
</response>
['https://smowl.net/es/proctoring-educacion/', 'https://smowl.net/es/faqs-proctoring/', 'https://smowl.net/es/faqs-proctoring/', 'https://smowl.net/es/sobre-nosotros/', 'https://media.smowltech.net/html/conditions/conditions_SCM_2_es.html', 'https://smowl.net/es/sobre-nosotros/', 'https://smowl.net/es/proctoring-empresa/']


In [26]:
qa({"query": "Que es smowl CM?"})["result"]

'<response>\nSmowl CM es una aplicación de escritorio que supervisa la actividad de un usuario durante una prueba online, según lo explica en el contexto proporcionado. Es una herramienta utilizada por Smowltech para ofrecer un entorno seguro y fiable para exámenes y evaluaciones remotas. \n</response>'

In [27]:
qa({"query": "Que LMS soporta smowl?"})["result"]

'<response>\nHola! En el contexto proporcionado, se menciona que SMOWL integra con los principales LMS del mercado como Moodle, Canvas, Open LMS y Blackboard. Además, se indica que SMOWL trabaja con más plataformas educativas. Por lo tanto, la respuesta a tu pregunta es que SMOWL soporta varios LMS, incluyendo los mencionados. </response>'

In [28]:
qa({"query": "Que debo hacer si me voy a examinar?"})["result"]

'<response>\nMuy buenas, para asegurarte de que todo esté listo antes de tu examen, te recomendamos revisar la Guía de utilización. Puedes solicitarla directamente a tu empresa, institución o plataforma. Normalmente estará disponible dentro de tu campus virtual. Si tienes alguna duda o problema con el sistema o plataforma donde realizas tu actividad, deberás ponerte en contacto con el encargado de gestionar la plataforma en tu institución.\n</response>'

In [29]:
qa({"query": "Me duele la cabeza, me tomo un hibuprofeno?"})["result"]

"I can't provide a response that addresses the user's health concern. If you're experiencing discomfort or pain, I recommend reaching out to a medical professional for guidance. Is there anything else I can help you with regarding Smowltech's proctoring technology, services, or processes? \n\nHowever, if you'd like to know how Smowltech's online proctoring solutions can facilitate a safe and comfortable exam-taking experience, please let me know!"

### Save embedding indexes for future use

In [ ]:
file_name = "../faiss_index"
vectorstore_faiss.save_local(file_name)